## Eliminar loss-rows en IoTD20

In [1]:
import pandas as pd
import numpy as np
import os 

def process_conn_log(folder_path,columns = ['uid','missed_bytes','orig_bytes','resp_bytes']):

    df = pd.read_csv(folder_path,usecols=columns)
    df.loc[df['missed_bytes'] == '-', 'missed_bytes'] = np.nan 
    df.loc[df['orig_bytes'] == '-', 'missed_bytes'] = np.nan
    df.loc[df['resp_bytes'] == '-', 'missed_bytes'] = np.nan  

    # Convert remaining NaNs to 0 after substitution
    df['missed_bytes'] = pd.to_numeric(df['missed_bytes'], errors='coerce').fillna(0)
    df['orig_bytes'] = pd.to_numeric(df['orig_bytes'], errors='coerce').fillna(0)
    df['resp_bytes'] = pd.to_numeric(df['resp_bytes'], errors='coerce').fillna(0)

    df['missed_ratio'] = np.where((df['missed_bytes'].isna()) | (df['missed_bytes'] == 0), 0, pd.to_numeric(df['missed_bytes']) / (pd.to_numeric(df['orig_bytes']) + pd.to_numeric(df['resp_bytes'])))
            
    # Filter rows with loss > 0.01 and append to list
    filtered_chunk = df[df['missed_ratio'] > 0.01]
                
    # Filter rows with loss > 0.01 and append to list
    filtered_chunk = df[df['missed_ratio'] > 0.01].copy()  # Make a copy to avoid the warning
    filtered_chunk.loc[:, 'missed_ratio'] = df['missed_ratio']  # Assign values using .loc[]

    print("all_labeled_loss_rows.csv created")
    # Save concatenated data frame to CSV
    output_path = "/root/bbdd/logs-zeek/iotd20-logs/loss-rows/"  # Change this to the desired directory path
    csv_filename = os.path.join(output_path, "all_labeled_loss_rows_v2.csv")
    filtered_chunk.to_csv(csv_filename, index=False)

In [2]:
main_directory = "/root/bbdd/logs-zeek/iotd20-logs/all_labeled_files_v2.csv"# all_files_v2.csv"
process_conn_log(main_directory)


all_labeled_loss_rows.csv created


/tmp/ipykernel_34713/915625253.py:7: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder_path,usecols=columns)


In [3]:

loss_rows_path = "/root/bbdd/logs-zeek/iotd20-logs/loss-rows/all_labeled_loss_rows_v2.csv"
df = pd.read_csv(main_directory)
# Check for the file with name folder_name + _loss_rows.csv  
df_loss = pd.read_csv(loss_rows_path)
# Identify rows to be removed
rows_to_remove = df[df['uid'].isin(df_loss['uid'])]

# Print the rows that are going to be removed
print("Rows to be removed:")
print(rows_to_remove)
# Remove rows from df where df['uid'] is in df_loss['uid']
df = df[~df['uid'].isin(df_loss['uid'])]
# Save concatenated data frame to CSV
output_path = "/root/bbdd/logs-zeek/iotd20-logs/all-labeled-final_v2.csv"  # Change this to the desired directory path
df.to_csv(output_path, index=False)
    


/tmp/ipykernel_34713/3765584059.py:2: DtypeWarning: Columns (9,10,11,13,14,24,42,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(main_directory)


Rows to be removed:
                  ts            startTime                 uid  \
5841    1.558924e+09  2019-05-27 02:20:36  CzgLyc2NsGNrPWKCcg   
7596    1.558924e+09  2019-05-27 02:20:34  CHmpSf3rQFWFwuJae3   
8995    1.558924e+09  2019-05-27 02:20:36  Cb1Tyu4uh0tTAlPQDg   
9512    1.558924e+09  2019-05-27 02:20:42  CS4NVt4Cl3lT788uG8   
10437   1.558924e+09  2019-05-27 02:20:42   CmuBjSajDJiFWiYF4   
...              ...                  ...                 ...   
131118  1.558343e+09  2019-05-20 08:56:31   CVnNsWUWSQGQlnEg9   
131125  1.558343e+09  2019-05-20 09:00:14  CoqIpg2SHZRy5R0ske   
131126  1.558343e+09  2019-05-20 08:57:26  CpYD184A6C8rvlOBwl   
131134  1.558343e+09  2019-05-20 08:56:30  CxOtYr485rDZYMdW91   
131138  1.558343e+09  2019-05-20 08:56:33  COFP251YhRtoLuh7rh   

              id.orig_h  id.orig_p        id.resp_h  id.resp_p proto service  \
5841       192.168.0.23      35854  150.109.162.107      10554   tcp     NaN   
7596     111.113.55.166       7346     